DUPLA: Gustavo Pierre e Victor Habib

TEMA: Nutella

# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

In [1649]:
%matplotlib inline
import pandas as pd
import numpy as np
import math as math
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from emoji import UNICODE_EMOJI

In [1650]:
nutella = pd.read_excel("tweets_nutella.xlsx")

# Limpar tweets

In [1651]:
def limpo(tweet):
    
    tweets_limpos = []
    
    
        
    tweet=tweet.replace('@', '')

    tweet=tweet.replace('.', '')

    tweet=tweet.replace(',', '')

    tweet=tweet.replace('rt ', '')

    tweet=tweet.replace(':', '')

    tweet=tweet.replace('_', '')

    tweet=tweet.replace('+', '')

    tweet=tweet.replace('-', '')

    tweet=tweet.replace('(', '')

    tweet=tweet.replace(')', '')

    tweet=tweet.replace('/', '')

    tweet=tweet.replace(';', '')

    tweet=tweet.replace('>', '')

    tweet=tweet.replace('<', '')

    tweet=tweet.replace('=', '')

    tweet=tweet.replace('#', '')

    tweet=tweet.replace('!', '')

    tweet=tweet.replace('*', '')

    tweet=tweet.replace('&', '')

    tweet=tweet.replace('$', '')

    tweet=tweet.replace('}', '')

    tweet=tweet.replace('{', '')

    tweet=tweet.replace('£', '')

    tweet=tweet.replace('[', '')

    tweet=tweet.replace(']', '')

    tweet=tweet.replace('?', '')

    tweet=tweet.replace('"', '')

    tweets_limpos.append(tweet)

    return tweet


In [1652]:
nutella["Treinamento"] = nutella["Treinamento"].apply(limpo)


Separação dos emojis

In [1653]:
emoji = []
nutella
for tweet in nutella["Treinamento"]:
    nada=''
    for palavra in tweet:
        if palavra in UNICODE_EMOJI:
            nada=nada+' '+palavra+' '
        else:
            nada+=palavra
    emoji.append(nada)
    
nutella["Treinamento"]=emoji


In [1654]:
nutella.Avaliação=nutella.Avaliação.astype("category")
nutella.Avaliação.cat.categories=("Relevante", "Irrelevante")

# Separar tweets em: relevante e irrelevante

IRRELEVANTE

In [1655]:
irrelevantes_counts=nutella["Treinamento"][nutella.Avaliação=='Irrelevante'].count()
irrelevantes_counts

129

In [1656]:
irrelevantes=nutella["Treinamento"][nutella.Avaliação=='Irrelevante']


RELEVANTE

In [1657]:
relevantes_counts=nutella["Treinamento"][nutella.Avaliação=='Relevante'].count()
relevantes_counts

171

In [1658]:
relevantes=nutella["Treinamento"][nutella.Avaliação=='Relevante']


TOTAL

In [1659]:
total=irrelevantes_counts+relevantes_counts
total

300

PROBABILIDADE DOS TWEETS SEREM RELEVANTES OU IRRELEVANTES

In [1660]:
p_tweet_irrelevante=irrelevantes_counts/total
p_tweet_relevante=relevantes_counts/total

print('Probabilidade do tweet ser irrelevante: {0:.2f}%'.format(p_tweet_irrelevante*100))
print('Probabilidade do tweet ser relevante: {0:.2f}%'.format(p_tweet_relevante*100))



Probabilidade do tweet ser irrelevante: 43.00%
Probabilidade do tweet ser relevante: 57.00%


# Contar palavras

IRRELEVANTE

In [1680]:
palavras_irrelevantes=[]

for linha in irrelevantes:
    separacao=linha.split()
    for palavra in separacao:
        palavras_irrelevantes.append(palavra)


print('Quantidade de palavras que aparecem nos tweets classificados como irrelevantes COM repetiçaõ: {0}'.format(len(palavras_irrelevantes)))
print('')




print('')
print('len de irrelevantes: {0}'.format(len(palavras_irrelevantes)))

Quantidade de palavras que aparecem nos tweets classificados como irrelevantes COM repetiçaõ: 2199


len de irrelevantes: 2199


In [1682]:
conta_palavras_irrelevantes=pd.Series(palavras_irrelevantes).value_counts()

#print('\nContagem de palavras que aparecem nos tweets classificados como irrelevantes')
print('\n possibilidade de irrelevantes: {0}'.format(len(conta_palavras_irrelevantes)))

#conta_palavras_irrelevantes


 possibilidade de irrelevantes: 938


RELEVANTE

In [1683]:
palavras_relevantes=[]
for linha in relevantes:
    separacao=linha.split()
    for palavra in separacao:
        palavras_relevantes.append(palavra)
        
print('Quantidade de palavras que aparecem nos tweets classificados como relevantes COM repetiçaõ: {0}'.format(len(palavras_relevantes)))
print('')
        

print('')
print('len de relevantes: {0}'.format(len(palavras_relevantes)))

Quantidade de palavras que aparecem nos tweets classificados como relevantes COM repetiçaõ: 2544


len de relevantes: 2544


In [1684]:
conta_palavras_relevantes=pd.Series(palavras_relevantes).value_counts()

#print('\nContagem de palavras que aparecem nos tweets classificados como relevantes')
print('\npossibilidade de relevantes: {0}'.format(len(conta_palavras_relevantes)))
#conta_palavras_relevantes


possibilidade de relevantes: 775


TOTAL

In [1686]:
contagem_palavras_total=pd.Series(palavra_total).value_counts()
#print("\nContagem de palavras que aparecem em ambos os tweets")
#contagem_palavras_total

In [1687]:
palavra_total = []

for e in palavras_irrelevantes:
        palavra_total.append(e)
for y in palavras_relevantes:
        palavra_total.append(y)


conta_palavras_total
len(palavra_total)

4743

# Probabilidades

- P(Palavra|Relevante) = (ocorrencia da palavra + 1) / (len(relevante) + possibilidade de palavras)
- P(Palavra|Irreelevante) = (ocorrencia da palavra + 1) / (len(irrelevante) + possibilidade de palavras)

- P(Palavra|Relevante) = (conta_palavras_relevantes[palavra] + 1) / (len(palavras_relevantes) + len(contagem_palavras_total))
- P(Palavra|Irrelevante) = (conta_palavras_irrelevantes[palavra] + 1) / (len(palavras_irrelevantes) + len(contagem_palavras_total))

In [1667]:
def classificador_palavras(palavra):
    

    if palavra in palavra_total:
        
        if palavra in palavras_relevantes and palavra in palavras_irrelevantes:
            prob_relevante= (conta_palavras_relevantes[palavra] + 1) / (len(palavras_relevantes) + len(contagem_palavras_total))
            prob_irrelevante = (conta_palavras_irrelevantes[palavra] + 1) / (len(palavras_irrelevantes) + len(contagem_palavras_total))
           
        
        elif palavra in palavras_irrelevantes:
            prob_irrelevante=(conta_palavras_irrelevantes[palavra] + 1) / (len(palavras_irrelevantes) + len(contagem_palavras_total))
            prob_relevante = (0 + 1) / (len(palavras_relevantes) + len(contagem_palavras_total))
        
        elif palavra in palavras_relevantes:
            prob_relevante=(conta_palavras_relevantes[palavra] + 1) / (len(palavras_relevantes) + len(contagem_palavras_total))
            prob_irrelevante =  (0 + 1) / (len(palavras_irrelevantes) + len(contagem_palavras_total))
    
    else:
        prob_relevante = (0 + 1) / (len(palavras_relevantes) + len(contagem_palavras_total))
        prob_irrelevante =  (0 + 1) / (len(palavras_irrelevantes) + len(contagem_palavras_total))
        
    return[prob_relevante, prob_irrelevante]


#print(classificador_palavras('nutella'))

- P(Tweet|Relevante) = P(Palavra_1|Relevante) * P(Palavra_2|Relevante) * P(Palavra_3|Relevante) * ... * P(Palavra_n|Relevante)
- P(Tweet|Irrelevante) = P(Palavra_1|Irrelevante) * P(Palavra_2|Irrelevante) * P(Palavra_3|Irrelevante) * ... * P(Palavra_n|Irrelevante)

In [1691]:
def classificador_tweet(tweet):
    probabilidade_tweet_relevante=1
    probabilidade_tweet_irrelevante=1
    limpa=limpo(tweet)
    separa_palavra=limpa.split()
    for palavra in separa_palavra: 
        probabilidades=classificador_palavras(palavra)
        
        probabilidade_tweet_relevante=probabilidades[0]*probabilidade_tweet_relevante
        probabilidade_tweet_irrelevante=probabilidades[1]*probabilidade_tweet_irrelevante
        
    if probabilidade_tweet_relevante > probabilidade_tweet_irrelevante:
        return 'Relevante'
    
    elif probabilidade_tweet_relevante < probabilidade_tweet_irrelevante:
        return 'Irrelevante'
    


In [1692]:
nutella_teste=pd.read_excel("tweets_nutella.xlsx", "Teste")

In [1693]:
nutella_teste.Avaliação=nutella_teste.Avaliação.astype("category")
nutella_teste.Avaliação.cat.categories=("Relevante", "Irrelevante")


In [1695]:
probs_teste=[]
for tweet in nutella_teste["Teste"]:
    probs=classificador_tweet(tweet)
    probs_teste.append(probs)


In [1696]:
nutella_teste["Classificados testes"]=pd.Series(probs_teste)
nutella_teste.head()

,Teste,Avaliação,Classificados testes
0,rt @arttyoongi: só fui eu que vi ou vocês tamb...,Irrelevante,Irrelevante
1,@quefilhodaputa nutella é ruim,Relevante,Relevante
2,@spunk182 @esp_interativo esse termo é usado d...,Irrelevante,Irrelevante
3,@arqtricolor isso aí! nada de arena nutella 😎!...,Irrelevante,Irrelevante
4,rt @clubefollowback: já temos até bolsonarista...,Irrelevante,Irrelevante


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [1697]:
acertos=0
erros=0
lista = []
i=0
    
for x in nutella_teste.Avaliação:
    lista.append(x)
    
while i < len(probs_teste):
    if probs_teste[i]==lista[i]:
        acertos+=1
        i+=1
    else:
        erros+=1
        i+=1
        

print('Acertos: {0}. Probabilidade de acerto: {1}%'.format(acertos, acertos/200*100))
print('Erros: {0}. Probabilidade de erro: {1}%'.format(erros, erros/200*100))

Acertos: 179. Probabilidade de acerto: 89.5%
Erros: 21. Probabilidade de erro: 10.5%


In [1698]:
pv = nutella_teste[(nutella_teste["Avaliação"]=="Relevante")&(nutella_teste["Classificados testes"]=="Relevante")]

nv = nutella_teste[(nutella_teste["Avaliação"]=="Irrelevante")&(nutella_teste["Classificados testes"]=="Irrelevante")]

nf = nutella_teste[(nutella_teste["Avaliação"]=="Relevante")&(nutella_teste["Classificados testes"]=="Irrelevante")]

pf = nutella_teste[(nutella_teste["Avaliação"]=="Irrelevante")&(nutella_teste["Classificados testes"]=="Relevante")]


In [1699]:
prob_positivo_verdadeiro = len(pv["Avaliação"])/len(nutella_teste["Avaliação"])
print("Porcentagem de positivo verdadeiro: {0:.2f}%".format(prob_positivo_verdadeiro*100))

Porcentagem de positivo verdadeiro: 59.00%


In [1700]:
prob_negativos_verdadeiros = len(fv["Avaliação"])/len(nutella_teste["Avaliação"])
print("Porcentagem de negativo verdadeiro: {0:.2f}%".format(prob_negativos_verdadeiros*100))

Porcentagem de negativo verdadeiro: 30.50%


In [1701]:
prob_negativos_falsos = len(nf["Avaliação"])/len(nutella_teste["Avaliação"])
print("Porcentagem de negativo falso: {0:.2f}%".format(prob_negativos_falsos*100))

Porcentagem de negativo falso: 2.00%


In [1702]:
prob_positivo_falso = len(pf["Avaliação"])/len(nutella_teste["Avaliação"])
print("Porcentagem de positivo falso: {0:.2f}%".format(prob_positivo_falso*100))

Porcentagem de positivo falso: 8.50%


___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


Baseando-se na análise qualitativa das medidas obtidas, o classificador implementado pode ser considerado confiável. Os tweets referentes a nutella, em geral, apresentam duas contextualizações. Uma delas faz referência ao produto, e a outra é utilizada na forma de adjetivo. Isso ocorre, pois o termo nutella foi adotado pela sociedade associado a algo que remete a modernidade, e no termo mais popular, "frescura". Nesse sentido, a palavra nutella é acompanhada implicita ou explicitamente da palavra "raiz", definida como o oposto: antiguidade e jeito bruto de ser ou agir. Em suma, como essas contextualizações são completamente independendes e irrelacionáveis, algumas palavras-chave que estão presentes nos tweets classificados como irrelevantes, não se encontram nos tweets classificados como relevantes, o que aumenta as chances de sucesso do classificador.

Com relação ao tratamento de mensagens com dupla negação e sarcasmo pode-se dizer que não houve muitas ocorrências desse tipo de tweet, tanto na planilha de treinamento, quanto na de teste. Entretanto, das poucas vezes que apareceu, consideramos todas as frases no sentido literal, desconsiderando qualquer forma de ironia. É inegável que isso prejudica a leitura e classificação dos tweets, mas diferente dos seres humanos, a máquina ainda não é 100% eficaz quando frases conotativas ou que apresentem sentidos inversos são dadas a elas para interpretarem.

Pensando em um plano de expansão, é valido dizer que a base está praticamente concluída. O classificador se encontra com uma probabilidade de acerto de 89,5%, apresentando um grande potencial de melhora. Dessa forma, acreditamos que manter o financimento do projeto pode trazer benefícios tanto aos criadores financiados quanto aos financiadores. Para maximizar o sucesso do classificador, algumas breves implementações podem ser introduzidas, como: maior filtragem dos tweets (limpar mais caracteres indesejados) e mais quantidades de tweets. Assim, a manutenção do financiamento permitiria que avançássemos no projeto.